##Ephemeris Generation##

This notebook is intended to test various methods of generating the positions and velocities of moving objects, including testing the accuracy of the resulting values calculated by interpolating between the points calculated by an n-body integrator. 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Read some orbital parameters.
import pandas
orbitfile = 'pha20141031.des'

orbits = pandas.read_table(orbitfile, sep='\s', engine='python')
orbits = orbits.to_records()

In [7]:
# Set up to run OpenOrb to generate ephemerides.
from itertools import repeat
import pyoorb as oo

oo.pyoorb.oorb_init(ephemeris_fname="")

1

Set up the times at which to calculate 'rough' positions. 
We will interpolate between these times to find the positions at the actual time of a visit. 

In [8]:
timestep = 2.0 / 24.0  # in days  -- previous value 
timestart = 49353
nyears = 1 # years
timeend = timestart + 365 * nyears + 1.0
times = np.arange(timestart, timeend + timestep/2.0, timestep)
# For pyoorb, we need to tag times with timescales;
# 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI
ephTimes = np.array(zip(times, repeat(4, len(times))), dtype='double', order='F')
print ephTimes.shape

(4393, 2)


Python oorb element format: 
        # Translate orbital elements into array that pyoorb will like.
        # PyOrb wants ::
        # 0: orbitId  (cannot be a string)
        # 1 - 6: orbital elements, using radians for angles
        # 7: element type code, where 2 = cometary - means timescale is TT, too
        # 8: epoch
        # 9: timescale for the epoch; 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI
        # 10: magHv
        # 11: G
so we have to do a little translating from the orbits DataFrame to the elements we want in this array. 

In [1]:
def packOorbElem(sso):
    """
    Convert an array of orbital elements into the array OpenOrb needs as input.
    'sso' can be the orbital elements of a single object or of multiple objects.
    """
    elem_type = np.zeros(len(sso['q'])) + 2
    epoch_type = np.zeros(len(sso['q'])) + 3
    # Also, the orbitID has to be a float, rather than a string, so substitute if needed.
    objid = None
    if '!!ObjID' in sso.dtype.names:
        objid = '!!ObjID'
    elif 'objid' in sso.dtype.names:
        objid = 'objid'
    if (objid is not None) and (isinstance(sso[objid][0], float) or isinstance(sso[objid][0], int)):
        orbids = sso[objid]
    else:
        orbids = np.arange(0, len(sso['q']), 1)
    # Search for appropriate 'keywords' for some other orbital parameters that vary in different input formats
    BigOmega = 'Omega/node'
    if 'node' in sso.dtype.names:
        BigOmega = 'node'
    omega = 'omega/argperi'
    if 'argperi' in sso.dtype.names:
        omega = 'argperi'
    tperi = 't_p'
    if 'timeperi' in sso.dtype.names:
        tperi = 'timeperi'
    inc = 'i'
    if 'inc' in sso.dtype.names:
        inc = 'inc'
    epoch = 't_0'
    if 'epoch' in sso.dtype.names:
        epoch = 'epoch'
    magH = 'magHv'
    if 'H' in sso.dtype.names:
        magH = 'H'
    if magH not in sso.dtype.names:
        Hval = np.zeros(len(sso['q'])) + 20.
    else:
        Hval = sso[magH]
    if 'phaseV' not in sso.dtype.names:
        phaseV = np.zeros(len(sso['q'])) + 0.15
    else:
        phaseV = sso['phaseV']
    
    # Convert to format for pyoorb, INCLUDING converting inclination, node, argperi to RADIANS
    oorbArray = np.column_stack((orbids, sso['q'], sso['e'], np.radians(sso[inc]),
                                 np.radians(sso[BigOmega]), np.radians(sso[omega]),
                                 sso[tperi], elem_type, sso[epoch], epoch_type, 
                                 Hval, phaseV))                                            
    return oorbArray

In [11]:
def packOorbElem2(sso):
    oorbelems = [sso['!!ObjID'], sso['q'], sso['e'], np.radians(sso['i']), np.radians(sso['Omega/node']), 
             np.radians(sso['omega/argperi']), sso['t_p'],  2, sso['t_0'], 3, sso['magHv'], 0.15]
    oorbelems = np.column_stack(oorbelems)
    return oorbelems

In [ ]:
print nyears, ' (years of ephemerides)'
for sso in orbits[0:1]:
    print sso
    oorbelems = packOorbElem2(sso)
    oorbephs, err = oo.pyoorb.oorb_ephemeris(in_orbits = oorbelems, in_obscode=807, in_date_ephems=ephTimes)